In [1]:
import typing

import pandas
import transformers

import cltrier_lib

/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODELS: typing.Dict[str, str] = {
    "base": "meta-llama/Llama-3.2-3B-Instruct", 
    "adapter": "simon-muenker/Llama-3.2-3B-Instruct-OSN-replies"
}
LEANINGS: typing.List[str] = ["left", "neutral", "right"]

In [3]:
INSTRUCTION = cltrier_lib.inference.schemas.Message(role="system", content="You are a social media user with a political {leaning} leaning. Respond to the following Tweet:")

In [4]:
SAMPLES: typing.List[str] = [
    "Die FDP Ostwestfalen-Lippe spricht sich für die Verlängerung der Laufzeiten der Kernkraftwerke aus.",
    "Seit 10 Tagen sind wir aus der #Atomkraft ausgestiegen. Das Licht brennt trotzdem zuverlässig und unser Land ist sicherer und freier geworden.",
    "Berlin ohne Autos - das könnte so schön sein!  So viel Platz, so viel Ruhe, so gute Luft.",
    "Wer pauschal über 'Sozialtourismus' und 'kleine Paschas' spricht, der kann keinen Führungsanspruch für das moderne Deutschland begründen!",
    "Wie lange wird die Diskussion ums #Tempolimit wohl noch gehen bis das Tempolimit endlich kommt?",
]

In [5]:
pipelines: typing.Dict[str, transformers.Pipeline] = {
    label: transformers.pipeline("text-generation", slug, device=f"cuda:{n}")
    for n, (label, slug) in enumerate(MODELS.items())
}

pipelines["adapter"].model.load_adapter(MODELS["adapter"])

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.77it/s]


In [6]:
responses = []

for idx, post in enumerate(SAMPLES):
    for leaning in LEANINGS:
        for model, pipeline in pipelines.items():
            chat = cltrier_lib.inference.schemas.Chat(messages=[
                INSTRUCTION.format_content(leaning=leaning),
                cltrier_lib.inference.schemas.Message(role="user", content=post)
            ])

            reply = pipeline(
                pipeline.tokenizer.apply_chat_template(chat, tokenize=False), 
                max_new_tokens=128,
                return_full_text=False
            )[0]["generated_text"].split("\n\n")[1]

            responses.append(
                dict(
                    id=idx,
                    leaning=leaning, 
                    model=model, 
                    post=post,
                    reply=reply
                )
            )
        

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pa

In [7]:
pandas.set_option('display.max_colwidth', None)

(
    pandas.DataFrame(responses)
    .set_index(["id", "leaning", "model"])
    .style.set_properties(**{'text-align': 'left'})
)